In [134]:
import requests
from secrets_config import api_key
import pandas as pd
# api_key = os.environ.get("api_key")

In [135]:
# Variables Note: these are case-sensitive
from_symbol = 'USD'
to_symbol = 'RUB'

In [132]:
def extract_fx_rates(
    from_symbol:str,
    to_symbol:str,
    api_key:str
)->pd.DataFrame:
    base_url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={api_key}'
    r = requests.get(base_url)
    response_data = r.json()
    df = pd.DataFrame(response_data['Time Series FX (Daily)']).transpose().reset_index()
    df = df.rename(columns={ df.columns[0]: "Date" })
    df['from'] = f'{from_symbol}'
    df['to'] = f'{to_symbol}'
    return df


In [133]:
extract_fx_rates(from_symbol=from_symbol, to_symbol=to_symbol, api_key=api_key)

,Date,1. open,2. high,3. low,4. close,from,to
0,2022-09-28,58.15000,59.19200,56.87300,57.88420,USD,RUB
1,2022-09-27,58.48880,59.40000,57.27500,58.89590,USD,RUB
2,2022-09-26,57.05000,58.86190,56.00000,58.48980,USD,RUB
3,2022-09-23,59.87250,60.23500,55.67500,56.87000,USD,RUB
4,2022-09-22,61.58480,61.60670,58.25000,59.89990,USD,RUB
...,...,...,...,...,...,...,...
95,2022-05-18,65.67500,65.72420,61.77500,64.73920,USD,RUB
96,2022-05-17,63.87500,65.68300,62.87500,65.47500,USD,RUB
97,2022-05-16,64.75000,64.87500,62.75000,63.87500,USD,RUB
98,2022-05-13,65.07500,66.10730,62.08350,64.50000,USD,RUB


In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.dialects import postgresql

def load(
        df: pd.DataFrame, 
        load_target:str, 
        load_method:str="upsert",
        target_database_engine=None,
        target_table_name:str=None
    )->None:
    """
    Load dataframe to database. 
    """
    import os 
    # create target table if not exist    meta = MetaData()
    usd_rub_fx = Table(
        target_table_name, meta, 
        Column("Date", String, primary_key=True),
        Column("1. open", String),
        Column("2. high", String),
        Column("3. low", String),
        Column("4. close", String),
        Column("from", String),
        Column("to", String)
    )
    meta.create_all(target_database_engine) # creates table if it does not exist 
    insert_statement = postgresql.insert(stock_price_tesla_table).values(df.to_dict(orient='records'))
    upsert_statement = insert_statement.on_conflict_do_update(
        index_elements=['timestamp', 'exchange'],
        set_={c.key: c for c in insert_statement.excluded if c.key not in ['timestamp','exchange']})
    target_database_engine.execute(upsert_statement)

In [139]:
import os
api_key = os.environ.get("api_key")
db_user = os.environ.get("db_user")
db_password = os.environ.get("db_password")
db_server_name = os.environ.get("db_server_name")
db_database_name = os.environ.get("db_database_name")

In [140]:
api_key

In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from sqlalchemy.schema import CreateTable 

# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)
engine = create_engine(connection_url)

# load database 
load(
    df=df_transformed,
    load_target="database", 
    load_method="upsert", 
    target_database_engine=engine,
    target_table_name="weather"
)

return True 